In [1]:
import pandas as pd
from obp.dataset import OpenBanditDataset
import os
import plotly.express as px
import numpy as np
import string
import plotly.graph_objects as go
import math
from plotly.subplots import make_subplots

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score

# Import visualization functions from our custom module
from visualizations import plot_bar_chart, plot_distribution_with_cdf

print(os.getcwd())

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

/Users/armandoordoricadelatorre/Documents/U of T/PhD/PhD Research/OBP_Replication


### Helper functions

In [2]:

def remap_user_features(df, feature_cols):
    """
    Map hash values in user_feature_N to short readable codes like A1, B1, ...
    """
    df_copy = df.copy()
    mapping_dicts = {}

    for col in feature_cols:
        # Extract the feature index (N from 'user_feature_N')
        feature_idx = col.split("_")[-1]
        uniques = df[col].dropna().unique()

        # Build codes A{N}, B{N}, C{N}...
        codes = [f"{letter}{feature_idx}" for letter in string.ascii_uppercase[:len(uniques)]]
        mapping = dict(zip(uniques, codes))

        df_copy[col] = df[col].map(mapping)
        mapping_dicts[col] = mapping

    return df_copy, mapping_dicts

### Import logged data from `all.csv` and `item_context.csv`

In [3]:

# BTS / ALL sample
log_df   = pd.read_csv("zr-obp/obd/bts/all/all.csv", index_col=0)
items_df = pd.read_csv("zr-obp/obd/bts/all/item_context.csv", index_col=0)

print("log_df shape:", log_df.shape)
print("items_df shape:", items_df.shape)

print("\nlog_df columns:")
print(log_df.columns.tolist()[:40])  # peek first ~40 col names

print("\nfirst 5 log rows:")
print(log_df.head())

print("\nfirst 5 item rows:")
print(items_df.head())

log_df shape: (10000, 89)
items_df shape: (80, 5)

log_df columns:
['timestamp', 'item_id', 'position', 'click', 'propensity_score', 'user_feature_0', 'user_feature_1', 'user_feature_2', 'user_feature_3', 'user-item_affinity_0', 'user-item_affinity_1', 'user-item_affinity_2', 'user-item_affinity_3', 'user-item_affinity_4', 'user-item_affinity_5', 'user-item_affinity_6', 'user-item_affinity_7', 'user-item_affinity_8', 'user-item_affinity_9', 'user-item_affinity_10', 'user-item_affinity_11', 'user-item_affinity_12', 'user-item_affinity_13', 'user-item_affinity_14', 'user-item_affinity_15', 'user-item_affinity_16', 'user-item_affinity_17', 'user-item_affinity_18', 'user-item_affinity_19', 'user-item_affinity_20', 'user-item_affinity_21', 'user-item_affinity_22', 'user-item_affinity_23', 'user-item_affinity_24', 'user-item_affinity_25', 'user-item_affinity_26', 'user-item_affinity_27', 'user-item_affinity_28', 'user-item_affinity_29', 'user-item_affinity_30']

first 5 log rows:
           

### Remapping categorical features to readable categories

In [4]:
user_feature_cols = [c for c in log_df.columns if c.startswith("user_feature")]
log_df_readable, mappings = remap_user_features(log_df, user_feature_cols)

print("Sample remapped features:")
display(log_df_readable[user_feature_cols].head())

print("\nMappings used:")
for feat, mapping in mappings.items():
    print(f"{feat}: {mapping}")

Sample remapped features:


,user_feature_0,user_feature_1,user_feature_2,user_feature_3
0,A0,A1,A2,A3
1,A0,B1,B2,B3
2,A0,A1,C2,A3
3,A0,A1,A2,B3
4,A0,A1,C2,B3



Mappings used:
user_feature_0: {'81ce123cbb5bd8ce818f60fb3586bba5': 'A0', 'cef3390ed299c09874189c387777674a': 'B0', '4ae385d792f81dde128124a925a830de': 'C0'}
user_feature_1: {'03a5648a76832f83c859d46bc06cb64a': 'A1', '2d03db5543b14483e52d761760686b64': 'B1', '6ff54aa8ff7a9dde75161c20a3ee4231': 'C1', 'f1c2d6a32ec39249160cf784b63f4c6f': 'D1', '8b50621825ffd909dd8d8317d366271f': 'E1'}
user_feature_2: {'7bc94a2da491829b777c49c4b5e480f2': 'A2', '2723d2eb8bba04e0362098011fa3997b': 'B2', 'c2e4f76cdbabecd33b8c762aeef386b3': 'C2', '719dab53a7560218a9d1f96b25d6fa32': 'D2', '9b2d331c329ceb74d3dcfb48d8798c78': 'E2', '302deff13f835d731df1c842eed95971': 'F2', '9f4e8271d3d3014af5f35124c2de5082': 'G2', '7ae37150e596e6e8f19e27a06bd4d359': 'H2', 'c7cce49040b6630e9b5484dfcc0e6cd1': 'I2'}
user_feature_3: {'c39b0c7dd5d4eb9a18e7db6ba2f258f8': 'A3', '9bde591ffaab8d54c457448e4dca6f53': 'B3', '05b76f5e97e51128862059ac7df9e42a': 'C3', 'f97571b9c14a786aab269f0b427d2a85': 'D3', '06128286bcc64b6a4b0fb7bc0328fe17'

In [5]:
log_df_readable.head()

,timestamp,item_id,position,click,propensity_score,user_feature_0,user_feature_1,user_feature_2,user_feature_3,user-item_affinity_0,user-item_affinity_1,user-item_affinity_2,user-item_affinity_3,user-item_affinity_4,user-item_affinity_5,user-item_affinity_6,user-item_affinity_7,user-item_affinity_8,user-item_affinity_9,user-item_affinity_10,user-item_affinity_11,user-item_affinity_12,user-item_affinity_13,user-item_affinity_14,user-item_affinity_15,user-item_affinity_16,user-item_affinity_17,user-item_affinity_18,user-item_affinity_19,user-item_affinity_20,user-item_affinity_21,user-item_affinity_22,user-item_affinity_23,user-item_affinity_24,user-item_affinity_25,user-item_affinity_26,user-item_affinity_27,user-item_affinity_28,user-item_affinity_29,user-item_affinity_30,user-item_affinity_31,user-item_affinity_32,user-item_affinity_33,user-item_affinity_34,user-item_affinity_35,user-item_affinity_36,user-item_affinity_37,user-item_affinity_38,user-item_affinity_39,user-item_affinity_40,user-item_affinity_41,user-item_affinity_42,user-item_affinity_43,user-item_affinity_44,user-item_affinity_45,user-item_affinity_46,user-item_affinity_47,user-item_affinity_48,user-item_affinity_49,user-item_affinity_50,user-item_affinity_51,user-item_affinity_52,user-item_affinity_53,user-item_affinity_54,user-item_affinity_55,user-item_affinity_56,user-item_affinity_57,user-item_affinity_58,user-item_affinity_59,user-item_affinity_60,user-item_affinity_61,user-item_affinity_62,user-item_affinity_63,user-item_affinity_64,user-item_affinity_65,user-item_affinity_66,user-item_affinity_67,user-item_affinity_68,user-item_affinity_69,user-item_affinity_70,user-item_affinity_71,user-item_affinity_72,user-item_affinity_73,user-item_affinity_74,user-item_affinity_75,user-item_affinity_76,user-item_affinity_77,user-item_affinity_78,user-item_affinity_79
0,2019-11-24 00:00:17.004101+00:00,79,2,0,0.087125,A0,A1,A2,A3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-11-24 00:00:19.715857+00:00,14,1,0,0.006235,A0,B1,B2,B3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-11-24 00:01:04.303227+00:00,18,2,0,0.061300,A0,A1,C2,A3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-11-24 00:01:11.571162+00:00,28,1,0,0.019430,A0,A1,A2,B3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-11-24 00:02:41.811768+00:00,65,2,0,0.019375,A0,A1,C2,B3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## EDA 

### Histogram on number of rows per day 

In [6]:
# Ensure proper datetime dtype (keeps timezone info)
log_df = log_df_readable.copy()
log_df['timestamp'] = pd.to_datetime(log_df['timestamp'], utc=True, errors='coerce')
log_df = log_df.loc[log_df['timestamp'].notna()]

# convert to your local timezone (example: Toronto)
log_df['timestamp'] = log_df['timestamp'].dt.tz_convert('America/Toronto')

# # Count rows per day, including days with zero rows
daily_counts = (
    log_df
    .set_index('timestamp')
    .resample('D')
    .size()
    .rename('rows')
    .reset_index()
)

# Use the plotting function
fig = plot_bar_chart(
    data=daily_counts,
    x_col='timestamp',
    y_col='rows',
    x_title='Date',
    y_title='Rows',
    title='Number of Rows per Day',
    height=500,
    show_values=True,
    value_format='{:,}'  # thousands separator
)

fig.show()


### Counting user features vs user-item features

In [7]:
user_feature_cols = [c for c in log_df_readable.columns if c.startswith('user_feature_')]
affinity_cols = [c for c in log_df_readable.columns if c.startswith('user-item_affinity_')]

user_feature_cols, len(affinity_cols)

(['user_feature_0', 'user_feature_1', 'user_feature_2', 'user_feature_3'], 80)

### Cardinality per feature

In [8]:
feat_summary = []
for c in user_feature_cols:
    s = log_df_readable[c]
    vc = s.value_counts(dropna=False)
    feat_summary.append({
        'feature': c,
        'n_unique': s.nunique(dropna=True),
        'n_missing': int(s.isna().sum()),
        'top_value': None if vc.empty else vc.index[0],
        'top_count': 0 if vc.empty else int(vc.iloc[0]),
        'top_share': 0.0 if vc.empty else float(vc.iloc[0] / len(s)),
    })
pd.DataFrame(feat_summary).sort_values('n_unique', ascending=False)

,feature,n_unique,n_missing,top_value,top_count,top_share
2,user_feature_2,9,0,B2,2439,0.2439
3,user_feature_3,9,0,B3,3667,0.3667
1,user_feature_1,5,0,A1,8322,0.8322
0,user_feature_0,3,0,A0,8123,0.8123


In [9]:
log_df_readable['item_id'].nunique(), items_df.shape[0]

(80, 80)

In [10]:
log_df_readable.groupby(['user_feature_0']).size()

user_feature_0
A0    8123
B0    1808
C0      69
dtype: int64

In [11]:
# Loop through all user features and create distribution plots
for feature_col in user_feature_cols:
    # Extract feature number for the title
    feature_num = feature_col.split('_')[-1]
    
    fig = plot_distribution_with_cdf(
        data=log_df_readable,
        column_name=feature_col,
        title=f'Distribution of User Feature {feature_num} (PDF + CDF)',
        height=600,
        sort_by='count'  # Sort by count (descending)
    )
    fig.show()


Distribution Summary: user_feature_0
Total observations: 10,000
Unique values: 3

Top 5 values:
user_feature_0  count  percentage  cumulative_pct
            A0   8123       81.23           81.23
            B0   1808       18.08           99.31
            C0     69        0.69          100.00




Distribution Summary: user_feature_1
Total observations: 10,000
Unique values: 5

Top 5 values:
user_feature_1  count  percentage  cumulative_pct
            A1   8322       83.22           83.22
            B1    854        8.54           91.76
            D1    681        6.81           98.57
            C1    138        1.38           99.95
            E1      5        0.05          100.00




Distribution Summary: user_feature_2
Total observations: 10,000
Unique values: 9

Top 5 values:
user_feature_2  count  percentage  cumulative_pct
            B2   2439       24.39           24.39
            C2   2411       24.11           48.50
            A2   1569       15.69           64.19
            E2   1514       15.14           79.33
            D2   1407       14.07           93.40




Distribution Summary: user_feature_3
Total observations: 10,000
Unique values: 9

Top 5 values:
user_feature_3  count  percentage  cumulative_pct
            B3   3667       36.67           36.67
            A3   3523       35.23           71.90
            D3   1164       11.64           83.54
            E3    913        9.13           92.67
            C3    549        5.49           98.16

